In [1]:
import pandas as pd
import re
import Levenshtein
from pandarallel import pandarallel
import threading
import os

pandarallel.initialize(progress_bar=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [2]:
def is_null(s: str) -> bool:
    if(len(s) == 0):
        return True
    s = re.sub(r'(.)\1+', r'\1', s)
    null_words = ['отсутствует', 'нет', 'нету', 'na']
    min_dist = min(Levenshtein.distance(s, i) for i in null_words)
    if(len(s) <= 5) :
        return min_dist <= 1
    else:
        return min_dist <= 3

In [3]:
translit_dict = {
    "sch": "щ", "sh": "ш", "ch": "ч", "yu": "ю", "ya": "я", "yo": "ё",
    "ye": "е", "zh": "ж", "ts": "ц", "ee": "и", "ay": "эй", "iy": "ий",
    
    "a": "а", "b": "б", "c": "ц", "d": "д", "e": "е", "f": "ф", "g": "г", 
    "h": "х", "i": "и", "j": "й", "k": "к", "l": "л", "m": "м", "n": "н",
    "o": "о", "p": "п", "q": "к", "r": "р", "s": "с", "t": "т", "u": "у",
    "v": "в", "w": "в", "x": "кс", "y": "ы", "z": "з",
}

regex_pattern = re.compile("|".join(re.escape(key) for key in sorted(translit_dict.keys(), key=len, reverse=True)))


In [4]:
def translit_callback(match):
    return translit_dict[match.group(0)]

def transliterate_eng_to_rus(text):
    return regex_pattern.sub(translit_callback, text)

In [5]:
def erase_spec(s: str) -> str:
    return re.sub(r'[^a-zа-я ]', '', s)

def normalize_name(name: str):
    name = erase_spec(name.lower())
    if(is_null(name)):
        return ''
    
    return transliterate_eng_to_rus(name)



In [6]:
def normalize_year(y: str):
    if len(y) == 2:
        if int(y) < 25:
            return '20' + y
        else:
            return '19' + y
    elif len(y) == 3:
        if y[0] == '1':
            return '19' + y[1:]
        elif y[0] == '0':
            return '2' + y 
        else:
            return '1' + y
    else:
        return y

In [7]:
def normalize_date(date: str): 
    if '-' in date:
        li = date.split('-')
        li[0] = normalize_year(li[0])
        return '-'.join(li)
    else:
        return normalize_year(date)

In [8]:
def normalize_phone(phone: str):
    phone = re.sub(r'[^+0-9]', '', phone)
    if(phone.startswith('+7') or phone.startswith('7') and len(phone) > 9):
        phone = re.sub(r'\+?7', '8', phone, 1)
    return phone

In [9]:
def normalize_addr(addr: str):
    return re.sub(r'[^а-яa-z0-9/ ]', '', addr.lower())

In [10]:
def normalize1():
    chunk_size = 2**17
    
    try:
        os.remove('main1_normalize.csv')
    except Exception:
        pass
    
    main1_iterator = pd.read_csv('main1.csv', chunksize=chunk_size, keep_default_na=False)
    output_file = 'main1_normalize.csv'

    write_header = True

    for chunk in main1_iterator:
        
        chunk['full_name'] = chunk['full_name'].apply(normalize_name)
        chunk.rename(columns = {'full_name': 'name'}, inplace=True)
        del chunk['sex']
        chunk['birthdate'] = chunk['birthdate'].apply(normalize_date)
        chunk['phone'] = chunk['phone'].apply(normalize_phone)
        chunk['address'] = chunk['address'].apply(normalize_addr)

        chunk.to_csv(output_file, mode='a', index=False, header=write_header)
        write_header = False


In [11]:
def normalize2():
    chunk_size = 2**18

    try:
        os.remove('main2_normalize.csv')
    except Exception:
        pass

    main2_iterator = pd.read_csv(
        'main2.csv', chunksize=chunk_size, keep_default_na=False)
    output_file = 'main2_normalize.csv'

    write_header = True

    for chunk in main2_iterator:

        chunk['first_name'] = chunk['first_name'].apply(normalize_name).str.cat(
            chunk['middle_name'].apply(normalize_name), sep=' ').str.cat(
                chunk['last_name'].apply(normalize_name), sep=' ')
        chunk.rename(columns = {'first_name': 'name'}, inplace=True)
        del chunk['middle_name']
        del chunk['last_name']
        chunk['birthdate'] = chunk['birthdate'].apply(normalize_date)
        chunk['phone'] = chunk['phone'].apply(normalize_phone)
        chunk['address'] = chunk['address'].apply(normalize_addr)

        chunk.to_csv(output_file, mode='a', index=False, header=write_header)
        write_header = False

In [12]:
def normalize3():
    chunk_size = 2**17

    try:
        os.remove('main3_normalize.csv')
    except Exception:
        pass

    main3_iterator = pd.read_csv('main3.csv', chunksize=chunk_size, keep_default_na=False)
    output_file = 'main3_normalize.csv'

    write_header = True

    for chunk in main3_iterator:
        
        chunk['name'] = chunk['name'].apply(normalize_name)
        chunk['birthdate'] = chunk['birthdate'].apply(normalize_date)
        del chunk['sex']

        chunk.to_csv(output_file, mode='a', index=False, header=write_header)
        write_header = False
        

In [13]:
def normalize():
    print("LOOL")
    p1 = threading.Thread(target=normalize1)
    p2 = threading.Thread(target=normalize2)
    p3 = threading.Thread(target=normalize3)
    p1.start()
    p2.start()
    p3.start()
    p1.join()
    p2.join()
    p3.join()

In [14]:
normalize()

LOOL


In [21]:
import tabulate
import pandas as pd

ch=pd.read_csv("main2_normalize.csv",chunksize=50).get_chunk()
print(tabulate.tabulate(list(ch.drop(columns=['uid']), showindex=False)))

['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', ' ', ' ', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', ' ', ' ', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', ' ', ' ', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '\n', 'м', 'а', 'м', 'а', 'с', 'а', 'и', 'д', ' ', 'к', 'а', 'р', 'л', 'и', 'т', 'о', ' ', 'б', 'а', 'л', 'к', 'и', 'б', 'а', 'е', 'в', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '1', '9', '5', '4', '-', '0', '6', '-', '1', '5', ' ', ' ', '8', '0', '2', '0', '6', '4', '5', '0', '5', '2', '6', ' ', ' ', 'г', ' ', 'у', 'р', 'у', 'с', 'м', 'а', 'р', 'т', 'а', 'н', ' ', 'п', 'е', 'р', ' ', 'ш', 'м', 'и'